In [49]:
import sys


# import os to capture file in the directory
import os
from os.path import join


# import re to clean text, import np and pandas to frame the text data
import re, numpy as np, pandas as pd
from pprint import pprint


# NLTK Stop words
import nltk
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['paper', 'research', 'study', 'literature', 'article'
                   'author' , 'purpose', 'develop',  'apply', 'make', 'set',
                   'assume', 'investigate', 'explore', 'propose', 'analyze', 'examine', 'analysis',
                   'result', 'discuss', 'aim', 'demonstrate', 'illustrate', 'show', 'find', 'suggest', 
                   'significant', 'important', 'addition', 'finally'])


# Gensim for model construction, spacy for lemmatization
import gensim, spacy
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

import en_core_web_sm

# Enable logging for gensim - optional
import logging, warnings

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

In [67]:
content = pd.read_excel(join('F:\\Projects\\SCF_LR\\1_data', 'SCF_Modelling_Topic_5_3_3_1.xlsx'), sheet_name='Topic_5_3_3_1').fillna('')
# content = pd.read_excel(join('F:\\Projects\\SCF_LR\\1_data', 'SCFpapers_SortByJournal_clean.xlsx'), sheet_name='Sheet1').fillna('')
df = pd.DataFrame(content['Article'] + ". " + content['Abstract'] + ". " +content['Keywords'])
df.columns = ["content"]

In [68]:
## # Tokenize and Clean-up using gensim’s simple_preprocess()

In [69]:
# delete unwanted signs and sparate sentence into single word
def sent_to_words(sentences):
    for sent in sentences:
#        sent = re.sub('Â©', '', sent)  # remove Â©
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)

In [70]:
# Convert to list
data = df.content.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['mixture', 'inventory', 'model', 'of', 'lost', 'sale', 'and', 'back', 'order', 'with', 'stochastic', 'lead', 'time', 'demand', 'on', 'permissible', 'delay', 'in', 'payments', 'it', 'is', 'seen', 'that', 'the', 'trade', 'credit', 'period', 'has', 'an', 'important', 'role', 'in', 'real', 'business', 'world', 'in', 'this', 'article', 'an', 'inventory', 'model', 'has', 'been', 'developed', 'by', 'considering', 'stochastic', 'lead', 'time', 'demand', 'with', 'lead', 'time', 'crashing', 'cost', 'here', 'also', 'to', 'get', 'the', 'impact', 'between', 'credit', 'period', 'and', 'lead', 'time', 'lead', 'time', 'dependent', 'credit', 'period', 'has', 'been', 'considered', 'in', 'this', 'model', 'considering', 'partial', 'back', 'order', 'the', 'effect', 'of', 'lost', 'sale', 'has', 'been', 'included', 'under', 'the', 'above', 'considerations', 'an', 'inventory', 'model', 'has', 'been', 'optimized', 'in', 'the', 'parlance', 'of', 'infinite', 'time', 'horizon', 'here', 'three', 'objective', 'fu

In [71]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=1) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=1)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = en_core_web_sm.load(disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!
print(data_ready[:1])

C:\Users\zhibd\Anaconda3\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['mixture', 'inventory_model', 'lost', 'sale', 'back', 'order', 'stochastic', 'lead_time', 'demand', 'payment', 'see', 'trade_credit', 'period', 'role', 'real', 'business', 'world', 'article', 'inventory_model', 'consider', 'stochastic', 'lead_time', 'demand', 'lead_time', 'crash', 'cost', 'also', 'get', 'impact', 'lead_time', 'lead_time', 'dependent', 'credit_period', 'consider', 'model', 'consider', 'partial', 'back', 'order', 'effect', 'lose', 'sale', 'include', 'consideration', 'inventory_model', 'optimize', 'parlance', 'infinite', 'time', 'horizon', 'objective', 'function', 'basis', 'position', 'credit_period', 'business', 'period', 'feasibility', 'model', 'different', 'sensitivity', 'respect', 'parameter', 'crash', 'cost', 'inventory_model', 'lead_time', 'stochastic', 'demand']]


In [72]:
# Find the optimal number of topics for LDA.

In [73]:
# Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus by terming Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

In [74]:
# Build normal LDA model

In [95]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [96]:
# Show dominant topic and its percentage contribution in each document

In [97]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.7428,"dcf, full, real, objective, discount, lead_tim...","[mixture, inventory_model, lost, sale, back, o..."
1,1,1.0,0.5148,"supplier, demand, retailer, offer, credit_peri...","[retailer, optimal, ordering_policy, order, si..."
2,2,1.0,0.9887,"supplier, demand, retailer, offer, credit_peri...","[level, credit_period, coordination, periodic,..."
3,3,3.0,0.8424,"price, function, optimal, pricing, also, retai...","[effect, two_echelon, trade_credit, pricing, i..."
4,4,1.0,0.6776,"supplier, demand, retailer, offer, credit_peri...","[retailer, supplier, supply_chain, model, trad..."
5,5,5.0,0.7484,"payment, trade_credit, order_quantity, conditi...","[lot, sizing, policy, defective, time_dependen..."
6,6,1.0,0.9908,"supplier, demand, retailer, offer, credit_peri...","[two_level, delay, payment, contract, supply_c..."
7,7,4.0,0.9889,"trade_credit, retailer, quantity, policy, cred...","[inventory_model, receipt, item, integrate, la..."
8,8,1.0,0.9222,"supplier, demand, retailer, offer, credit_peri...","[delay, payment, contract, pareto, improvement..."
9,9,5.0,0.8693,"payment, trade_credit, order_quantity, conditi...","[inventory_model, time_dependent, demand, shor..."


In [98]:
writer = pd.ExcelWriter(join('F:\\Projects\\SCF_LR\\1_data', 'SCFpapers_Classification_modelling_5_3_3_1.xlsx'))
df_dominant_topic.to_excel(writer,'Sheet1')
writer.save()

In [99]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

C:\Users\zhibd\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.198391  0.074777       1        1  33.135841
1      0.098456 -0.106676       2        1  26.377857
4     -0.094923  0.223684       3        1  21.178837
3      0.023324 -0.037815       4        1   9.407379
0     -0.142002 -0.084502       5        1   5.832410
2     -0.083246 -0.069468       6        1   4.067678, topic_info=     Category       Freq             Term      Total  loglift  logprob
term                                                                  
115   Default  28.000000            price  28.000000  30.0000  30.0000
88    Default  50.000000         supplier  50.000000  29.0000  29.0000
10    Default  39.000000           demand  39.000000  28.0000  28.0000
33    Default  99.000000          payment  99.000000  27.0000  27.0000
82    Default  57.000000         retailer  57.000000  26.0000  26.0000
273   Default  18.000000         quantity  18.000000  25.0000  25.0000
15    Default  16.000000         function  16.000000  24.0000  24.0000
69    Default  24.000000            offer  24.000000  23.0000  23.0000
117   Default  15.000000          problem  15.000000  22.0000  22.0000
413   Default   8.000000           permit   8.000000  21.0000  21.0000
71    Default  27.000000   order_quantity  27.000000  20.0000  20.0000
70    Default  39.000000          optimal  39.000000  19.0000  19.0000
9     Default  19.000000    credit_period  19.000000  18.0000  18.0000
29    Default  30.000000            order  30.000000  17.0000  17.0000
132   Default  25.000000        condition  25.000000  16.0000  16.0000
0     Default  16.000000             also  16.000000  15.0000  15.0000
54    Default  37.000000            delay  37.000000  14.0000  14.0000
56    Default  31.000000        determine  31.000000  13.0000  13.0000
21    Default  23.000000  inventory_model  23.000000  12.0000  12.0000
5     Default  14.000000         consider  14.000000  11.0000  11.0000
116   Default   8.000000          pricing   8.000000  10.0000  10.0000
316   Default  12.000000    cash_discount  12.000000   9.0000   9.0000
98    Default  16.000000           credit  16.000000   8.0000   8.0000
280   Default  11.000000   supplier_offer  11.000000   7.0000   7.0000
208   Default  30.000000             item  30.000000   6.0000   6.0000
44    Default  61.000000     trade_credit  61.000000   5.0000   5.0000
153   Default  10.000000          product  10.000000   4.0000   4.0000
398   Default   5.000000        warehouse   5.000000   3.0000   3.0000
412   Default   5.000000        otherwise   5.000000   2.0000   2.0000
79    Default  10.000000          present  10.000000   1.0000   1.0000
...       ...        ...              ...        ...      ...      ...
545    Topic6   0.741676              int   1.440559   2.5382  -5.0143
424    Topic6   0.567367          combine   1.265001   2.4003  -5.2822
430    Topic6   0.567367          instead   1.265001   2.4003  -5.2822
431    Topic6   0.567367              net   1.265001   2.4003  -5.2822
421    Topic6   0.567367           accept   1.265001   2.4003  -5.2822
433    Topic6   0.567367          specify   1.265001   2.4003  -5.2822
40     Topic6   0.898157              see   2.086217   2.3593  -4.8228
28     Topic6   0.884294         optimize   2.098412   2.3380  -4.8384
14     Topic6   0.441002      feasibility   1.136600   2.2553  -5.5341
6      Topic6   0.441002    consideration   1.136600   2.2553  -5.5341
24     Topic6   0.441002             lost   1.136600   2.2553  -5.5341
3      Topic6   0.441002            basis   1.136600   2.2553  -5.5341
38     Topic6   0.441002             role   1.136600   2.2553  -5.5341
16     Topic6   0.441002              get   1.136600   2.2553  -5.5341
516    Topic6   0.939155          concept   2.421073   2.2551  -4.7782
35     Topic6   0.441002         position   1.136600   2.2553  -5.5341
31     Topic6   0.441002         parlance   1.